In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("FirstApp").getOrCreate()

24/03/18 10:25:10 WARN Utils: Your hostname, miguels-MBP.local resolves to a loopback address: 127.0.0.1; using 192.168.100.17 instead (on interface en0)
24/03/18 10:25:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 10:25:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=FirstApp>

24/03/18 10:25:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df = spark.read.format("csv").option("header",True).option("inferSchema",True).load("./trip_yellow_taxi.csv")
# df.write.format("csv").option("header",True).save("example.csv")
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2017-07-01 00:06:25|  2017-07-01 00:10:50|              1|          1.2|         1|                 N|         249|          90|           1|        5.5|  0.5|    0.5|      1.35|         0.0|                  0.3|        8.15|
|       1| 2017-07-01 00:20:04|  2017-07-01 00:2

In [5]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



1. Fetch the record having VendorID as '2' AND tpep_pickup_datetime as '2017-10-01 00:15:30' AND tpep_dropoff_datetime as '2017-10-01 00:25:11' AND passenger_count as '1'

In [6]:
df.filter(df.VendorID=='2').filter(df.tpep_pickup_datetime=='2017-10-01 00:15:30')\
                           .filter(df.tpep_dropoff_datetime=='2017-10-01 00:25:11')\
                           .filter(df.passenger_count=='1')\
                           .show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2| 2017-10-01 00:15:30|  2017-10-01 00:25:11|              1|         2.17|         1|                 N|         141|         142|           1|        9.0|  0.5|    0.5|      2.06|         0.0|                  0.3|       12.36|
|       2| 2017-10-01 00:15:30|  2017-10-01 00:2

In [6]:
df.createOrReplaceTempView('taxi_data')

In [9]:
FILTER_QUERY = """
    SELECT
        *
    FROM
        taxi_data
    WHERE VendorID==2
        AND tpep_pickup_datetime=='2017-10-01 00:15:30'
        AND tpep_dropoff_datetime=='2017-10-01 00:25:11'
        AND passenger_count=='1'
"""

In [10]:
spark.sql(FILTER_QUERY).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2| 2017-10-01 00:15:30|  2017-10-01 00:25:11|              1|         2.17|         1|                 N|         141|         142|           1|        9.0|  0.5|    0.5|      2.06|         0.0|                  0.3|       12.36|
|       2| 2017-10-01 00:15:30|  2017-10-01 00:2

 2. Filter all the records having RatecodeID as 4.

In [11]:
df.filter(df.RatecodeID=='4').show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2017-12-01 00:42:45|  2017-12-01 01:04:59|              1|         23.3|         4|                 N|         265|          82|           3|      115.5|  0.5|    0.5|       0.0|         0.0|                  0.3|       116.8|
|       1| 2017-12-01 02:49:44|  2017-12-01 02:5

 3. Group By all the records based on payment type and find the count for each group. 

In [16]:
df.groupBy("payment_type").count().show()

+------------+-----+
|payment_type|count|
+------------+-----+
|           1|14322|
|           3| 2484|
|           4|  396|
|           2| 7452|
+------------+-----+



In [15]:
FILTER_QUERY = """
    SELECT
        extract(year from tpep_pickup_datetime), count(1)
    FROM
        taxi_data
    GROUP BY extract(year from tpep_pickup_datetime)
"""

In [16]:
spark.sql(FILTER_QUERY).show()

+---------------------------------------+--------+
|extract(year FROM tpep_pickup_datetime)|count(1)|
+---------------------------------------+--------+
|                                   2017|   24654|
+---------------------------------------+--------+



In [18]:
from pyspark.sql.functions import to_date, col

In [19]:
# Create new partition key
# add a new column to the DataFrame or replace an existing one if the name matches. 
# The new column is named p_date.
# to_date converts the tpep_pickup_datetime timestamp into a date, 
# removing any time information, and stores this in the new p_date column
df_sink = df.withColumn('p_date', to_date(col('tpep_pickup_datetime')))

In [22]:
df_sink.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|    p_date|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+
|       1| 2017-07-01 00:06:25|  2017-07-01 00:10:50|              1|          1.2|         1|                 N|         249|          90|           1|        5.5|  0.5|    0.5|      1.35|         0.0|                  0.3|        8.15|2017-07-01|
+---

In [23]:
# Write to local storage, if not done already
# Writes the DataFrame in the Parquet file format, which is a columnar storage format offering efficient data compression and encoding schemes. 
df_sink.write.partitionBy("p_date").mode("overwrite").parquet("datasets/yellow_taxis_daily")

### Let's create a real "dimension" table, for our RateCodeID
1. Standard rate
2. JFK
3. Newark
4. Nassau or Westchester 
5. Negotiated fare
6. Group ride

In [26]:
from pyspark.sql.types import StringType, StructField, StructType

In [33]:
# Add Rate Code IDs
data = [("1", "Standard rate"), ("2", "JFK"), ("3", "Newark"),("4","Nassau or Westchester "),("5","Negotiated fare"), ("6","Group ride")]

In [34]:
# Define schema, to ensure data types
schema = StructType([ \
    StructField("rate_code_id",StringType(),True), \
    StructField("RatecodeName",StringType(),True)
  ])

In [35]:
# Create Dataframe for Rate Codes
df_rate_codes = spark.createDataFrame(data=data,schema=schema)

In [36]:
# Show rates
df_rate_codes.show()

+------------+--------------------+
|rate_code_id|        RatecodeName|
+------------+--------------------+
|           1|       Standard rate|
|           2|                 JFK|
|           3|              Newark|
|           4|Nassau or Westche...|
|           5|     Negotiated fare|
|           6|          Group ride|
+------------+--------------------+



In [37]:
# Inner join example:
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["rate_code_id"], "inner").show(n=5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+-------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|rate_code_id| RatecodeName|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+-------------+
|       1| 2017-12-01 03:49:15|  2017-12-01 04:26:21|              2|          8.4|         1|                 N|         164|          37|           4|       31.5|  0.5|    0.5|       0.0|        5.76|   

In [40]:
# SQL Statement
spark.sql('''
          select RatecodeID, count(1)
          from taxi_data
          group by RatecodeID
          ''').show(n=5)

+----------+--------+
|RatecodeID|count(1)|
+----------+--------+
|         1|    3450|
|         3|      30|
|         5|      72|
|         4|   21018|
|         2|      84|
+----------+--------+



In [41]:
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["rate_code_id"], "left").where("RatecodeID is NULL").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|rate_code_id|RatecodeName|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+------------+
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+------

In [20]:
spark.stop()